In [51]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

load_dotenv()

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=500,
    return_messages=True
)

examples = [
    {
        "movie": "인터스텔라",
        "answer": "🚀⏳🌌"
    },
    {
        "movie": "라라랜드",
        "answer": "🎹💃💔"
    },
    {
        "movie": "기생충",
        "answer": "🏠🍜🔪"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}가 무슨 영화야?"),
    ("ai", "{answer}"),
])

fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", """너는 영화를 3개의 이모지로 평가하는 영화평론가야.

    역할:
    1. 영화 제목을 물어보면 → 위 예제의 스타일을 참고해서 이모지 3개로 답변
    2. 이전 대화를 묻는 질문 (예: "처음 뭐 물어봤어?", "몇 개 물어봤어?")
       → 아래 [실제 대화 기록]만 참고해서 답변해. 위의 예제는 절대 포함하지 마!
    """),
    MessagesPlaceholder(variable_name="history"),
    fewshot_chat_prompt,
    ("human", "{question}"),
])


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

In [52]:
invoke_chain("배트맨")

content='🦇🌃👊'


In [53]:
invoke_chain("슈퍼맨")

content='🦸\u200d♂️💥🌟'


In [54]:
invoke_chain("처음에 질문한 영화가 뭐야?")

content='배트맨'
